In [3]:
%matplotlib inline

In [5]:
import numpy as np
import scipy as sp
import scipy.sparse.linalg as linalg
import matplotlib.pyplot as plt

# Task 1: 1D PEC-PMC Cavity, Eigenmode formulation

### 0. Maxwell Equations (time domain)
$\nabla \times E = - \mu \frac{\partial H}{\partial t}$

$\nabla \times H = \epsilon \frac{\partial E}{\partial t}$

### 1. Wave Equations

$\Delta E = \mu \epsilon \frac{\partial^2 E}{\partial t^2}$

$\Delta H = \mu \epsilon \frac{\partial^2 H}{\partial t^2}$

Since there is no difference between the equations, we will use the wave equation for $E$

### 2. Time Domain $\rightarrow$ Frequency Domain

Using the Fourier transform we get:

$E(r, t) \rightarrow E(r, w)$

$\frac{\partial E}{\partial t} \rightarrow i\omega E$

Thus our wave equation transforms into:

$\Delta E = - \omega^2 \mu \epsilon E$

### 3. 1D Coordinates

$\frac{\partial^2 E_z}{\partial x^2} = \omega^2 \mu \epsilon E_z$

### 4. Discretization

$\frac{\partial {E_z}_i}{\partial x} \approx \frac{{E_z}_{i+1} - {E_z}_{i-1}}{\Delta x}$

$\frac{\partial^2 {E_z}_i}{\partial x^2} \approx \frac{\frac{\partial {E_z}_{i+\frac{1}{2}}}{\partial x} - \frac{\partial {E_z}_{i-\frac{1}{2}}}{\partial x}}{\Delta x} = \frac{\frac{{E_z}_{i+1} - {E_z}_{i}}{\Delta x} - \frac{{E_z}_{i} - {E_z}_{i-1}}{\Delta x}}{\Delta x} = \frac{{E_z}_{i+1} - 2 {E_z}_{i} + {E_z}_{i-1}}{\Delta x^2}$

Which gives us:

$\frac{{E_z}_{i+1} - 2 {E_z}_{i} + {E_z}_{i-1}}{\Delta x^2} = \omega^2 \mu_i \epsilon_i {E_z}_{i}$

${E_z}_{i+1} -(2 + \Delta x^2 \omega^2 \mu_i \epsilon_i){E_z}_{i} + {E_z}_{i-1} = 0$

${E_z}_{i+1} -2{E_z}_{i} + {E_z}_{i-1} = \Delta x^2 \omega^2 \mu_i \epsilon_i{E_z}_{i}$

$\frac{1}{\mu_i \epsilon_i \Delta x^2} ({E_z}_{i+1} -2{E_z}_{i} + {E_z}_{i-1}) = \omega^2 {E_z}_{i}$
### 5. Matrix form

$A_i = \frac{1}{\Delta x^2 \mu_i \epsilon_i}$

or just A if $\mu_i = \mu$ and $\epsilon_i = \epsilon$

$
%\begin{bmatrix}
%    A_{0} \\
%    A_{1} \\
%    \vdots \\
%    A_{n-1} \\    
%    A_{n}    
%\end{bmatrix}
A
\begin{bmatrix}
    -2    & 1      & 0      & \dots  & 0      \\
    1      & -2    & 1      & \dots  & 0      \\
    0      & 1      & -2    & \dots  & \vdots \\
    \vdots & \vdots & \vdots & \ddots & 1      \\
    0      & 0      & \dots  & 1      & -2
\end{bmatrix}
\begin{bmatrix}
    {E_z}_{0} \\
    {E_z}_{1} \\
    \vdots \\
    {E_z}_{n-1} \\    
    {E_z}_{n}    
\end{bmatrix}
= \omega^2 
\begin{bmatrix}
    {E_z}_{0} \\
    {E_z}_{1} \\
    \vdots \\
    {E_z}_{n-1} \\    
    {E_z}_{n}    
\end{bmatrix}
$

### 6. Boundary Conditions

#### PEC

${E_z}_{i} = 0$

#### PMC

${H_y}_{i} = 0$

$\nabla \times E = - \mu \frac{\partial H}{\partial t}$

$\frac{{E_z}_{i}}{\partial x}  = 0$

$\frac{{E_z}_{i}}{\partial x}  \approx \frac{{E_z}_{i+1} - {E_z}_{i-1}}{\Delta x}$

${E_z}_{i+1} = {E_z}_{i-1}$

# Implementation
### 0. Setup grid

In [1]:
n   = 100     # Num grid nodes
dx  = 1. / n  # Step size for overall size of 1

eps = 1.      #  Vacuum
mu  = 1.      #  Vacuum

A   = 1. / (dx**2 * mu * eps)

### 1. Build matrix

In [14]:
diag = np.ones(n) * -2 * A
up_diag = np.ones(n) * A
down_diag = np.ones(n) * A

#PEC
diag[0] = 0

#PMC
diag[-1] = 1 * A
down_diag[-1] =  -1 * A

M = sp.sparse.dia_matrix(([up_diag, diag, down_diag], [1, 0, -1]), [n,n])

In [15]:
kt = 2*sp.pi*1  # wave vector target to find resonance wavelength
k2, V = linalg.eigs(M, k=6, M=None, sigma=kt**2) 